In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import numpy as np
import gzip
import os
import anndata as ad
import scipy.spatial as scisp
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

/tmp/ipykernel_2393272/2461235975.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
sc.settings.n_jobs=20

# 1. Combine SR and LR data

In [3]:
# read the filtered ones
samples_dict = {"Mock":"Mock", "D3":"D3PI","D7":"D7PI", "D10":"D10PI",
                "D14":"D14PI","D21":"D21PI"}
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[samples_dict[sample]] = sc.read_h5ad("../output_files/objects/adata_"+ sample+ "_with_image_raw_counts_filtered_smear_removed.h5ad")
adata_dict

{'Mock': AnnData object with n_obs × n_vars = 10256 × 31053
     obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_neighbors_100_true'
     var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     obsm: 'spatial', 'spatial_distances',
 'D3PI': AnnData object with n_obs × n_vars = 18661 × 31053
     obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
     var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     obsm: 'spatial',
 'D7PI': AnnData object with n_obs × n_vars = 26692 × 31053
     obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_neighbors_100_true'
     var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [1]:
1+1

2

In [ ]:
adata_comb = ad.concat(adata_dict, axis=0, label='Sample')
adata_comb.obs['Index'] = adata_comb.obs['barcode'] + '_' + adata_comb.obs['Sample'].astype(str)
adata_comb.obs.index = adata_comb.obs['Index']
# add the previous label
adata_prev = sc.read_h5ad("../../../output_files/objects/adata_allTP_demul_1lev_meta_c2lTB_memsec_ovpCS_wRegion_raw.h5ad")
col_int = ['S_score','G2M_score','phase','Celltypes','Broad_celltypes','Refine_celltypes','CD4_CD8_type','Bcell_subtype_abun','Region_Leiden_0.7']
for col in col_int:
    if col == "Celltypes":
        new_col = "Celltypes_c2lLN"
    elif col == "Region_Leiden_0.7":
        new_col = "Region"
    elif col == "Bcell_subtype_abun":
        new_col = "Bcell_subtype_c2lHTA"
    else:
        new_col = col
    adata_comb.obs[new_col] = adata_prev.obs[col]
# add the csv
lr_meta = pd.read_csv("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/AllTP_meta_ovp.tsv")
# merge together
adata_comb.obs.index = adata_comb.obs['Index']
lr_meta.index = lr_meta['Index']
lr_col = lr_meta.columns.tolist()
lr_col = [col for col in lr_col if col not in ["barcode","Sample","Index"]]
lr_meta = lr_meta[lr_col]
# left_join
adata_comb.obs = pd.merge(adata_comb.obs, lr_meta, left_index=True, right_index = True, how = "left")

In [ ]:
# add membrane-secreted information for all CS
# read the ovp CS df
meta_mem_sec = pd.read_csv("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/membrane_secreted_check/AllTP_membrane_secreted_ovp_Concensus_reads.txt",
                        sep = "\t",index_col="Index")
meta_mem_sec.columns = [f"{col}_CS" for col in meta_mem_sec.columns]
meta_mem_sec.columns
# meta_mem_sec.head()
# merge
adata_comb.obs = pd.merge(adata_comb.obs, meta_mem_sec, left_index=True, right_index = True, how = "left")
adata_comb.obs

In [ ]:
# add membrane-secreted information for all CDR3
# read the ovp CDR3 df
meta_mem_sec = pd.read_csv("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/membrane_secreted_check/AllTP_membrane_secreted_ovp_Mixcr.txt",
                        sep = "\t",index_col="Index")
meta_mem_sec.columns = [f"{col}_CDR3" for col in meta_mem_sec.columns]
meta_mem_sec.columns
# meta_mem_sec.head()
# merge
adata_comb.obs = pd.merge(adata_comb.obs, meta_mem_sec, left_index=True, right_index = True, how = "left")
adata_comb.obs.columns

In [ ]:
umi_columns = [col for col in adata_comb.obs.columns if col.endswith('_umi')]
print(umi_columns)
# add a num_IGHG_umi
adata_comb.obs['num_IGHG_umi'] = (
    adata_comb.obs['num_IGHG1_umi'] + 
    adata_comb.obs['num_IGHG2B_umi'] + 
    adata_comb.obs['num_IGHG2C_umi'] + 
    adata_comb.obs['num_IGHG3_umi']
)

In [ ]:
# replace 0.0 to na
ct_list = ['num_clones_IGH','num_clones_IGK','num_clones_IGL']
for i in ct_list:
    adata_comb.obs[i].replace(0.0,np.nan,inplace=True)
adata_comb.obs['num_clones_IGpair_HK'] = (adata_comb.obs['num_clones_IGH'] * adata_comb.obs['num_clones_IGK'])
adata_comb.obs['num_clones_IGpair_HL'] = (adata_comb.obs['num_clones_IGH'] * adata_comb.obs['num_clones_IGL'])
adata_comb.obs["num_clones_IGpair"] = adata_comb.obs['num_clones_IGpair_HK'] + adata_comb.obs['num_clones_IGpair_HL']

In [ ]:
# BT pair
ct_list = ['num_clones_TRB','num_clones_TRA']
for i in ct_list:
    adata_comb.obs[i].replace(0.0,np.nan,inplace=True)
adata_comb.obs['num_clones_BTpair_HB'] = (adata_comb.obs['num_clones_IGH'] * adata_comb.obs['num_clones_TRB'])

In [ ]:
# pairs for TR
adata_comb.obs['num_clones_TRpair'] = (adata_comb.obs['num_clones_TRB'] * adata_comb.obs['num_clones_TRA'])
adata_comb.obs[['num_clones_IGpair_HK','num_clones_IGpair_HL','num_clones_IGpair','num_clones_TRpair','num_clones_BTpair_HB']]

In [ ]:
adata_comb.obs['num_clones_TRpair'].sum(0,skipna=True)
# Change specific nam
adata_comb.obs['Region'] = adata_comb.obs['Region'].astype(str)
adata_comb.obs.loc[adata_comb.obs['Region'] == "Trabeculae", 'Region'] = "Conduit"

In [ ]:
# save
adata_comb.write_h5ad("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/objects/adata_comb_allTP_whole_meta_raw.h5ad")
# to csv
adata_comb.obs.to_csv("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/adata_comb_allTP_whole_meta.csv")

In [ ]:
adata_comb